In [1]:
import os

In [2]:
%pwd

'c:\\Users\\deepak\\Kidney-Disease-Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\deepak\\Kidney-Disease-Classification-MLflow-DVC'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: str
    training_data: str
    mlflow_uri: str
    repo_owner: str
    repo_name: str
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/deepaku0222/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            repo_owner="deepaku0222",
            repo_name="Kidney-Disease-Classification-MLflow-DVC",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

    def log_into_mlflow(self):
        import dagshub
        dagshub.init(
            repo_owner=self.config.repo_owner,
            repo_name=self.config.repo_name,
            mlflow=True
        )

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    artifact_path="model",
                    registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, artifact_path="model")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-20 12:26:39,256: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-20 12:26:39,256: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-20 12:26:39,272: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 73s 8s/step - loss: 2.4905 - accuracy: 0.6043
[2025-08-20 12:27:53,569: INFO: common: json file saved at: scores.json]
[2025-08-20 12:27:54,635: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as deepaku0222

[2025-08-20 12:27:54,651: INFO: helpers: Accessing as deepaku0222]
[2025-08-20 12:27:55,815: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/deepaku0222/Kidney-Disease-Classification-MLflow-DVC "HTTP/1.1 200 OK"]
[2025-08-20 12:27:56,754: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "deepaku0222/Kidney-Disease-Classification-MLflow-DVC"

[2025-08-20 12:27:56,754: INFO: helpers: Initialized MLflow to track repo "deepaku0222/Kidney-Disease-Classification-MLflow-DVC"]


Repository deepaku0222/Kidney-Disease-Classification-MLflow-DVC initialized!

[2025-08-20 12:27:56,770: INFO: helpers: Repository deepaku0222/Kidney-Disease-Classification-MLflow-DVC initialized!]


2025/08/20 12:27:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-20 12:28:01,240: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\deepak\AppData\Local\Temp\tmpf2b9a7nk\model\data\model\assets
[2025-08-20 12:28:03,046: INFO: builder_impl: Assets written to: C:\Users\deepak\AppData\Local\Temp\tmpf2b9a7nk\model\data\model\assets]


c:\Users\deepak\anaconda3\envs\Kidney\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/08/20 12:28:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
